In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, trim, col, to_date, round

StatementMeta(, 0f010e2c-f630-41bb-bc37-ab51c7b15e9e, 11, Finished, Available, Finished)

In [10]:
# 1️⃣ Read Bronze
df = spark.read.format("delta").table("BronzeLakehouse.dbo.Orders")

# 2️⃣ Clean and select columns
resultant_df = df.select(
    col("Order_ID"),
    col("Customer_ID"),
    col("Product_ID"),
    to_date(col("Order_Date"), "M/d/yyyy").alias("Order_Date"),
    col("Quantity").cast("int").alias("Quantity"),
    col("Price").cast("double").alias("Price"),
    trim(col("Status")).alias("Status"),
    col("ingestion_time")
)


StatementMeta(, 0f010e2c-f630-41bb-bc37-ab51c7b15e9e, 12, Finished, Available, Finished)

In [11]:

# 3️⃣ Drop duplicates
resultant_df = resultant_df.dropDuplicates(["Order_ID", "Product_ID"])

# Add Order_Total column
resultant_df = resultant_df.withColumn("Order_Total", round(col("Quantity") * col("Price"), 2))

# 4️⃣ Add silver ingestion time
resultant_df = resultant_df.withColumn("silver_ingestion_time", current_timestamp())

# 5️⃣ Write to Silver
resultant_df.write.format("delta").mode("overwrite").saveAsTable("Slv_Orders")

StatementMeta(, 0f010e2c-f630-41bb-bc37-ab51c7b15e9e, 13, Finished, Available, Finished)